In [1]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [2]:
AF2_HOST = "http://localhost:8000"

In [3]:
url = f'{AF2_HOST}/v1/health/ready'
print(url)

http://localhost:8000/v1/health/ready


In [4]:
# Health Check
import requests

url = f'{AF2_HOST}/v1/health/ready'

headers = {
    "content-type": "application/json"
}
try:
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.ok:
        print("Request succeeded:", response.json())
    else:
        print("Request failed:", response.status_code, response.text)
except Exception as E:
    print("Request failed:", E)

Request succeeded: {'status': 'ready'}


In [ ]:
# Jackhmmer
import requests
import json

url = f'{AF2_HOST}/protein-structure/alphafold2/predict-structure-from-sequence'

sequence = "MNVIDIAIAMAI"  # Replace with the actual sequence value

headers = {
    "content-type": "application/json"
}

data = {
    "sequence": sequence,
    "databases": ["uniref90"],
    "selected_models": [1]
}

response = requests.post(url, headers=headers, data=json.dumps(data))

# Check if the request was successful
if response.ok:
    print("Request succeeded:", response.json())
else:
    print("Request failed:", response.status_code, response.text)

In [5]:
# Mmseqs2
import requests
import json

url = f'{AF2_HOST}/protein-structure/alphafold2/predict-structure-from-sequence'
sequence = "MNVIDIAIAMAI"  # Replace with the actual sequence value

headers = {
    "content-type": "application/json"
}

data = {
    "sequence": sequence,
    "databases": ["uniref90", "small_bfd", "mgnify"],
    "algorithm": "mmseqs2"
}
# check wall time
%time response = requests.post(url, headers=headers, data=json.dumps(data))

# Check if the request was successful
if response.ok:
    print("Request succeeded:", response.json())
else:
    print("Request failed:", response.status_code, response.text)

CPU times: user 70.7 ms, sys: 11.8 ms, total: 82.5 ms
Wall time: 19min 12s
Request succeeded: ['ATOM      1  N   MET A   1     -14.418   7.945 -15.143  1.00 62.16           N  \nATOM      2  H   MET A   1     -14.198   7.605 -16.069  1.00 62.16           H  \nATOM      3  H2  MET A   1     -13.929   8.814 -14.982  1.00 62.16           H  \nATOM      4  H3  MET A   1     -15.412   8.103 -15.067  1.00 62.16           H  \nATOM      5  CA  MET A   1     -13.993   6.946 -14.146  1.00 62.16           C  \nATOM      6  HA  MET A   1     -14.175   7.309 -13.135  1.00 62.16           H  \nATOM      7  C   MET A   1     -12.502   6.787 -14.326  1.00 62.16           C  \nATOM      8  CB  MET A   1     -14.738   5.618 -14.344  1.00 62.16           C  \nATOM      9  HB2 MET A   1     -14.518   5.216 -15.333  1.00 62.16           H  \nATOM     10  HB3 MET A   1     -14.391   4.907 -13.594  1.00 62.16           H  \nATOM     11  O   MET A   1     -12.068   6.048 -15.202  1.00 62.16           O  \nAT

In [6]:
result = response.json()

In [7]:
len(result)

5

In [8]:
result[0][:400]

'ATOM      1  N   MET A   1     -14.418   7.945 -15.143  1.00 62.16           N  \nATOM      2  H   MET A   1     -14.198   7.605 -16.069  1.00 62.16           H  \nATOM      3  H2  MET A   1     -13.929   8.814 -14.982  1.00 62.16           H  \nATOM      4  H3  MET A   1     -15.412   8.103 -15.067  1.00 62.16           H  \nATOM      5  CA  MET A   1     -13.993   6.946 -14.146  1.00 62.16          '

In [30]:
import os
output_dir = "~/.cache/pred_structures"
!mkdir -p {output_dir}

In [31]:
!ls ~/.cache

motd.legal-displayed  nim  pip	pred_structures


In [32]:
fp = os.path.join(output_dir, f"predicted_protein.pdb")
with open(fp, "w") as f:
    f.write(result[0])

In [33]:
!pip install py3dmol

Defaulting to user installation because normal site-packages is not writeable


In [36]:
!pip show py3dmol

Name: py3Dmol
Version: 2.4.2
Summary: An IPython interface for embedding 3Dmol.js views in Jupyter notebooks
Home-page: https://3dmol.org
Author: David Koes
Author-email: dkoes@pitt.edu
License: MIT
Location: /home/ubuntu/.local/lib/python3.10/site-packages
Requires: 
Required-by: 


In [37]:
import sys
sys.path.append("/home/ubuntu/.local/lib/python3.10/site-packages")

import py3Dmol  # Now try importing it

In [38]:
def load_protein(pdb_file_path, width=800, height=600):

    """
    Load a protein structure from a PDB file and display it using py3Dmol.
    pdb_file_path: str, path to the PDB file
    width: int, width of the viewer in pixels
    height: int, height of the viewer in pixels
    return: py3Dmol.view object
    """

    with open(pdb_file_path) as ifile:
        pdb_data = "".join([x for x in ifile])

    view = py3Dmol.view(width=width, height=height)
    view.addModelsAsFrames(pdb_data)

    for line in pdb_data.split("\n"):
        split = line.split()
        if len(split) == 0 or split[0] != "ATOM":
            continue
        # Assuming the B-factor is at position 10 (you may need to adjust this based on your PDB format)
        b_factor = float(split[10])
        if b_factor > 90:
            color = "blue"
        elif 70 <= b_factor <= 90:
            color = "cyan"
        elif 50 <= b_factor < 70:
            color = "yellow"
        else:
            color = "orange"

        # Atom serial numbers typically start from 1, hence idx should be used directly
        idx = int(split[1])

        # Style should be set per atom id
        view.setStyle({'model': -1, 'serial': idx}, {"cartoon": {'color': color}})
    view.zoomTo()
    return view

In [39]:
view = load_protein(
    pdb_file_path = fp,
    width=800,
    height=500
)
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.